In [1]:
import PyPDF2
import fitz
import os
import re

ho_viet_nam = [
    "Nguyễn", "Trần", "Lê", "Phạm", "Hoàng", "Huỳnh", "Phan", "Vũ", "Võ", "Đặng", "Bùi", "Đỗ", "Hồ", "Ngô", "Dương", "Lý", "Đinh", "Đoàn", "Trương", "Văn", "Tô", "Tạ", "Tăng", "Tống", "Quách", "Lâm", "Cao", "Mai", "Khương", "Hà", "Đào", "Hồng", "Chu", "Hứa", "Từ", "Thái", "Ôn", "Thạch", "Châu", "Mạc", "Đàm", "La", "Giang", "Vi", "Lương", "Khuất", "Chử", "Lục", "Bành", "Lưu", "Tường", "Nhâm", "Ân", "An", "Triệu", "Kiều", "Cổ", "Nghiêm", "Bạch", "Ninh", "Đường", "Thân", "Phùng", "Thịnh", "Trịnh", "Quang", "Cấn", "Lai", "Thi", "Yên", "Tôn", "Sử", "Việt", "Ngọc", "Hàn", "Kim", "Vĩnh", "Đắc", "Thiều", "Bảo", "Diệp", "Xà", "Lư", "Cung", "Nông", "Giáp", "Kha", "Chung", "Tào", "Ái", "Hùng", "Ngũ", "Tịch", "Sơn", "Cù", "Phương", "Quyền", "Tiêu", "Bế", "Trác", "Hán", "Hầu", "Vương"
] 

# def get_file_name(pdf_path):
#     with open(pdf_path, 'rb') as pdf_file:
#         pdf_reader = PyPDF2.PdfReader(pdf_file)
#         file_name = ''
#         page = pdf_reader.pages[0]
#         page_text = page.extract_text()
#         lines = page_text.split('\n')
        
#         for i in range(1, len(lines)):
#             if len(lines[i]) == 0:
#                 continue
#             words = lines[i].split()
#             if not words:
#                 continue
#             first_word = words[0]
#             if i > 1:
#                 if first_word in ho_viet_nam:
#                     break
#             file_name += lines[i] + ' '
#         file_name = file_name[:-1].replace(' ', '_')
#         max_length = 200
#         if len(file_name) > max_length:
#             file_name = file_name[:max_length]
#         return file_name + '.txt'


def get_file_name(pdf_path):
    doc = fitz.open(pdf_path)
    page = doc[0]
    text_blocks = page.get_text("dict")["blocks"]

    # 📌 Biến lưu text có cỡ chữ lớn nhất
    max_font_size = 0
    max_text = ""

    # 📌 Duyệt qua từng khối văn bản để tìm text có cỡ chữ lớn nhất
    for block in text_blocks:
        if "lines" in block:  # Kiểm tra nếu có dòng chữ
            for line in block["lines"]:
                for span in line["spans"]:  # "spans" chứa thông tin về font và kích thước chữ
                    text = span["text"].strip()
                    font_size = span["size"]

                    if font_size > max_font_size:  # Cập nhật nếu cỡ chữ lớn hơn
                        max_font_size = font_size
                        max_text = text
                    elif font_size == max_font_size:
                        max_text += " " + text
                    elif max_font_size - font_size < 5 and text != "":
                        max_text += " " + text
                    
    # Giới hạn độ dài của tên tệp
    max_text = max_text[:100]  # Giới hạn tên tệp tối đa 100 ký tự
    return re.sub(r'[<>:"/\\|?*]', '_', max_text) + '.txt' #thay thế các kí tự đặc biệt không thể lưu thành tên file


def pdf_to_text(pdf_path, output_txt):
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ''
        
        page = pdf_reader.pages[0]
        page_text = page.extract_text()
        if page_text:
            text += ' '.join(page_text.splitlines()) + ' '

    lowered_text = text.lower()  # Đưa về chữ thường để tránh lỗi tìm kiếm
    lowered_text = re.sub(r'\s+', ' ', lowered_text)  # Thay thế tất cả các khoảng trắng dài hơn ' ' thành ' '
    text = re.sub(r'\s+', ' ', text)
    start_idx = lowered_text.find('tóm t ắt')
    if start_idx == -1:
        start_idx = lowered_text.find('tóm tắ t')
        end_idx = lowered_text.find('từ khóa')

        if start_idx != -1 and end_idx != -1 and start_idx < end_idx:
            summary_text = text[start_idx + len('tóm tắ t'): end_idx].strip()
            with open(output_txt, 'w', encoding='utf-8') as txt_file:
                txt_file.write(summary_text)
            return True
    end_idx = lowered_text.find('từ khóa')

    if start_idx != -1 and end_idx != -1 and start_idx < end_idx:
        summary_text = text[start_idx + len('tóm t ắt'): end_idx].strip()
        with open(output_txt, 'w', encoding='utf-8') as txt_file:
            txt_file.write(summary_text)
        return True
    
    return None

if __name__ == "__main__":
    input_dir = '/Users/hoangducanh/Documents/Hoc o HUST/nhom_anh_minh/crawl-abstract-data/data/input/DHHD/'
    output_dir = '/Users/hoangducanh/Documents/Hoc o HUST/nhom_anh_minh/crawl-abstract-data/data/output/DHHD/'

    for filename in os.listdir(input_dir):
        if filename.endswith('.pdf'):
            try:
                pdf_path = os.path.join(input_dir, filename)
                output_txt = os.path.join(output_dir, get_file_name(pdf_path))
                result = pdf_to_text(pdf_path, output_txt)
                
                if result:
                    print(f"✅ PDF {filename} Extract summary textsuccessfully!")
                else:
                    print(f"❌ Summary not found in the PDF {filename}.")
            except:
                pass

# if __name__ == "__main__":
#     input_dir = '/Users/hoangducanh/Documents/Hoc o HUST/nhom_anh_minh/crawl-abstract-data/data/input/most/'
#     output_dir = '/Users/hoangducanh/Documents/Hoc o HUST/nhom_anh_minh/crawl-abstract-data/data/output/most/'

#     for i, filename in enumerate(os.listdir(input_dir), start=1):
#         if filename.endswith('.pdf'):
#             pdf_path = os.path.join(input_dir, filename)
#             output_txt = os.path.join(output_dir, f"output{i}.txt")
#             result = pdf_to_text(pdf_path, output_txt)
            
#             if result:
#                 print(f"✅ PDF {filename} Extract summary text successfully!")
#             else:
#                 print(f"❌ Summary not found in the PDF {filename}.")

❌ Summary not found in the PDF input383.pdf.
❌ Summary not found in the PDF input397.pdf.
❌ Summary not found in the PDF input368.pdf.
✅ PDF input340.pdf Extract summary textsuccessfully!
❌ Summary not found in the PDF input426.pdf.
❌ Summary not found in the PDF input432.pdf.


incorrect startxref pointer(3)


❌ Summary not found in the PDF input354.pdf.
❌ Summary not found in the PDF input181.pdf.
✅ PDF input195.pdf Extract summary textsuccessfully!
✅ PDF input142.pdf Extract summary textsuccessfully!
✅ PDF input156.pdf Extract summary textsuccessfully!
❌ Summary not found in the PDF input86.pdf.
✅ PDF input92.pdf Extract summary textsuccessfully!
✅ PDF input45.pdf Extract summary textsuccessfully!
✅ PDF input51.pdf Extract summary textsuccessfully!


incorrect startxref pointer(3)


❌ Summary not found in the PDF input79.pdf.


...

❌ Summary not found in the PDF input239.pdf.


incorrect startxref pointer(3)


❌ Summary not found in the PDF input205.pdf.


incorrect startxref pointer(3)


❌ Summary not found in the PDF input211.pdf.
❌ Summary not found in the PDF input74.pdf.
❌ Summary not found in the PDF input60.pdf.
✅ PDF input48.pdf Extract summary textsuccessfully!
❌ Summary not found in the PDF input173.pdf.
✅ PDF input167.pdf Extract summary textsuccessfully!
✅ PDF input198.pdf Extract summary textsuccessfully!
❌ Summary not found in the PDF input359.pdf.
❌ Summary not found in the PDF input371.pdf.
❌ Summary not found in the PDF input417.pdf.
❌ Summary not found in the PDF input403.pdf.
❌ Summary not found in the PDF input365.pdf.
